## Berechnungen von Merkmalen

In diesem Notebook ist es möglich den Mittelwert, Varianz und Standardabeichung für die aufgenommenen Messwerte    
der Android TM Fahrradwege Monitoring App zu berechnen. Zu einem Bild werden parallel Messwerte erfasst von dem Beschleunigungssensor und Magnetsensor des Smartphones, innerhalb eines Zeitfenster mit der festen Länge von 80ms siehe hiezu auch das Notebook "gatheringSequence" in dem Ordner datenvisualisierung. Bei dem Beschleunigungssensor sind dies die Messwerte   
der 3-Achsen X,Y sowie Z (Abb. 1). Desweiteren werden die Gier-Nick-Roll Winkel in rad in dem Zeitfenster erfasst.       
Es wird in dem Notebook auch die Gieränderung in rad berechnet. Die Ergebnisse können in eine Merkmal CSV Datei geschrieben werden.

![alt text][logo]

[logo]: https://developer.android.com/images/axis_device.png "Logo Title Text 2"
Abbildung 1: 3-Achsen Smartphone Sensor-Koordinatensystem   
Quelle: https://developer.android.com/images/axis_device.png

In [1]:
import pandas
import numpy as np
import numpy.testing as npt
import math
import os

In [2]:
featuresDf = pandas.read_csv("../daten/merkmaleRoh.csv")
featuresDf.describe()

,Zeitstempel in Unixzeit,Breitengrad,Laengengrad,Geschwindigkeit in km/h,Anzahl der Messwerte,Start des Zeitfensters in ns seit Start der JVM,Start der Messwerterfassung in ns seit Start der JVM,Start der Belichtung in ns seit Start der JVM,Belichtungszeit in ns,Letzter Zeitstempel der Messwerterfassung in ns seit Start der JVM,Stopp der Messwerterfassung in Unixzeit
count,8.972000e+03,8972.000000,8972.000000,8972.000000,8972.000000,8.972000e+03,8.972000e+03,8.972000e+03,8.972000e+03,8.972000e+03,8.972000e+03
mean,1.523022e+12,53.617044,10.171727,13.979099,19.128399,4.276143e+13,4.276136e+13,4.276147e+13,3.481218e+06,4.276157e+13,1.523022e+12
std,9.279927e+05,0.003492,0.017890,3.855479,0.635943,9.279929e+11,9.279923e+11,9.279933e+11,2.714879e+06,9.279926e+11,9.279926e+05
min,1.523021e+12,53.613083,10.137841,5.004000,5.000000,4.115732e+13,4.115729e+13,4.115735e+13,6.435000e+05,4.115747e+13,1.523021e+12
25%,1.523021e+12,53.613976,10.156262,11.015999,19.000000,4.192303e+13,4.192296e+13,4.192307e+13,1.912116e+06,4.192318e+13,1.523021e+12
50%,1.523022e+12,53.615353,10.171855,14.112001,19.000000,4.283243e+13,4.283235e+13,4.283247e+13,2.298216e+06,4.283257e+13,1.523022e+12
75%,1.523023e+12,53.620280,10.186731,16.920000,19.000000,4.358764e+13,4.358756e+13,4.358768e+13,4.127597e+06,4.358778e+13,1.523023e+12
max,1.523024e+12,53.625160,10.201205,24.875998,43.000000,4.426992e+13,4.426987e+13,4.426995e+13,2.999632e+07,4.427009e+13,1.523024e+12


In [3]:
featuresDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8972 entries, 0 to 8971
Data columns (total 18 columns):
Zeitstempel in Unixzeit                                               8972 non-null int64
Breitengrad                                                           8972 non-null float64
Laengengrad                                                           8972 non-null float64
Geschwindigkeit in km/h                                               8972 non-null float64
X-Achse Beschleunigungswerte in m/s^2                                 8972 non-null object
Y-Achse Beschleunigungswerte in m/s^2                                 8972 non-null object
Z-Achse Beschleunigungswerte in m/s^2                                 8972 non-null object
Gier Messwerte in rad                                                 8972 non-null object
Nick Messwerte in rad                                                 8972 non-null object
Roll Messwerte in rad                                                 89

In [4]:
# Zeigt die ersten 5 Reihen Beschleunigungssensordaten der X-Achse 
featuresDf['X-Achse Beschleunigungswerte in m/s^2'].head()

0    0.50696 0.60484 0.60654 -0.06632 -0.28282 0.81...
1    0.14722 -0.18861 -0.22743 -0.35041 -0.03508 -0...
2    0.18452 -0.28135 -0.13307 0.20008 0.48192 0.96...
3    0.54282 0.25046 0.00124 -0.22876 0.61386 0.583...
4    0.82189 -1.31905 -2.0818 -2.75329 -3.12193 -2....
Name: X-Achse Beschleunigungswerte in m/s^2, dtype: object

Die folgenden Spalten des Datenframes haben als Type ein Python object.   
X-Achse Beschleunigungswerte in m/s^2   
Y-Achse Beschleunigungswerte in m/s^2   
Z-Achse Beschleunigungswerte in m/s^2   
Gier Messwerte in rad    
Nick Messwerte in rad          
Roll Messwerte in rad                   
Zeitstempel der Messwerte in ns               
Implizit haben diese Spalten als Typ Strings. Aber in einem DataFrame werden Strings als    
Python objects erkannt. Um den Datentyp auf float zu casten wird aus den Spalten zunächst ein Liste erstellt.    
Welche dann wiederum als Numpy Array auf den Typ float konvertiert wird.   

In [5]:
accXList = featuresDf['X-Achse Beschleunigungswerte in m/s^2'].str.split(" ").tolist()

In [6]:
# Direkt mit dem Numpy Array ist es nicht möglich die Konvertierung durchzuführen, weil die Arrays von Typ object sind.   
pandas.DataFrame(featuresDf['X-Achse Beschleunigungswerte in m/s^2'].str.split(" ")).values

array([[list(['0.50696', '0.60484', '0.60654', '-0.06632', '-0.28282', '0.81578', '1.08174', '0.49772', '0.50552', '0.92548', '0.47996', '-0.16758', '-0.57834', '-0.40131', '-0.15241', '0.76687', '0.6902', '0.50626', '0.81881', '0.40996'])],
       [list(['0.14722', '-0.18861', '-0.22743', '-0.35041', '-0.03508', '-0.24251', '-0.22457', '-0.77718', '-0.85151', '-0.69645', '-0.08208', '0.14894', '0.59424', '0.47547', '0.39577', '0.73042', '0.53844', '0.12437', '-0.06898', '-0.02445'])],
       [list(['0.18452', '-0.28135', '-0.13307', '0.20008', '0.48192', '0.96789', '0.42196', '-0.91882', '-1.8842', '-1.17018', '-0.18524', '0.2043', '-0.25019', '-0.58315', '-0.9721', '-1.08406', '-0.98975', '0.31152', '0.9848'])],
       ...,
       [list(['0.05452', '-0.24745', '-0.44306', '-0.43098', '-0.16084', '-0.18989', '0.10864', '0.05635', '0.45887', '0.81153', '0.83317', '0.89645', '0.67126', '-0.13713', '-0.40077', '-0.19796', '0.34737', '0.43119', '0.45229', '0.10141'])],
       [list(['-0.0

In [7]:
accXList[1] # zeige Liste

['0.14722',
 '-0.18861',
 '-0.22743',
 '-0.35041',
 '-0.03508',
 '-0.24251',
 '-0.22457',
 '-0.77718',
 '-0.85151',
 '-0.69645',
 '-0.08208',
 '0.14894',
 '0.59424',
 '0.47547',
 '0.39577',
 '0.73042',
 '0.53844',
 '0.12437',
 '-0.06898',
 '-0.02445']

In [8]:
np.array(accXList[1]).dtype

dtype('<U8')

In [9]:
# Konvertiert die Python Objekte in der Liste zu den Type Float.
# Das Ergebnis wird in einem Dataframe geschrieben
accList = []
accList.append(pandas.DataFrame(np.array(accXList[0]).astype(float)).transpose())
for x in accXList[1:]:
    accList.append(pandas.DataFrame(np.array(x).astype(float)).transpose())

In [10]:
accXDf = pandas.concat(accList, ignore_index=True)

In [11]:
accXDf.head()

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,0.50696,0.60484,0.60654,-0.06632,-0.28282,0.81578,1.08174,0.49772,0.50552,0.92548,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.14722,-0.18861,-0.22743,-0.35041,-0.03508,-0.24251,-0.22457,-0.77718,-0.85151,-0.69645,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.18452,-0.28135,-0.13307,0.20008,0.48192,0.96789,0.42196,-0.91882,-1.88420,-1.17018,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.54282,0.25046,0.00124,-0.22876,0.61386,0.58309,0.42058,-0.19975,-0.81861,-1.95726,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.82189,-1.31905,-2.08180,-2.75329,-3.12193,-2.92650,0.20248,1.24999,0.54038,0.04931,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# Um mehr Spalten eines Panda DataFrames auszugeben
pandas.set_option('display.max_columns', 40)
accXDf.abs().T.mean().head(20)

0     0.543271
1     0.346206
2     0.642584
3     0.649016
4     1.266264
5     0.873327
6     0.371622
7     0.901189
8     0.481675
9     0.473023
10    0.552326
11    0.371389
12    0.581557
13    0.981865
14    0.560765
15    0.511505
16    0.389722
17    0.778411
18    2.004932
19    0.810889
dtype: float64

In [13]:
accXDf.shape

(8972, 43)

In [14]:
# Die Anzahl der Messerte muss immer gleich sein, ansonsten werden fehlende Messwerte
# mit NaN gefüllt.
featuresDf['Anzahl der Messwerte'][featuresDf['Anzahl der Messwerte'] > 22]

2233    27
2566    23
3824    24
4987    43
Name: Anzahl der Messwerte, dtype: int64

In [15]:
# ausführen wenn alle Daten die gleiche Ahnzahl an Messwerten haben sollen. 
# Dadurch werden Reihen gelöscht!
unwantedRows = featuresDf['Anzahl der Messwerte'][featuresDf['Anzahl der Messwerte'] != 20]
featuresDf.drop(featuresDf.index, axis=0)

,Zeitstempel in Unixzeit,Breitengrad,Laengengrad,Geschwindigkeit in km/h,X-Achse Beschleunigungswerte in m/s^2,Y-Achse Beschleunigungswerte in m/s^2,Z-Achse Beschleunigungswerte in m/s^2,Gier Messwerte in rad,Nick Messwerte in rad,Roll Messwerte in rad,Zeitstempel der Messwerte in ns,Anzahl der Messwerte,Start des Zeitfensters in ns seit Start der JVM,Start der Messwerterfassung in ns seit Start der JVM,Start der Belichtung in ns seit Start der JVM,Belichtungszeit in ns,Letzter Zeitstempel der Messwerterfassung in ns seit Start der JVM,Stopp der Messwerterfassung in Unixzeit


In [16]:
accYList = featuresDf['Y-Achse Beschleunigungswerte in m/s^2'].str.split(" ").tolist()
accList = []
accList.append(pandas.DataFrame(np.array(accYList[0]).astype(float)).transpose())
for x in accYList[1:]:
    accList.append(pandas.DataFrame(np.array(x).astype(float)).transpose())
accYDf = pandas.concat(accList, ignore_index=True)

In [17]:
accYDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8972 entries, 0 to 8971
Data columns (total 43 columns):
0     8972 non-null float64
1     8972 non-null float64
2     8972 non-null float64
3     8972 non-null float64
4     8972 non-null float64
5     8971 non-null float64
6     8971 non-null float64
7     8971 non-null float64
8     8971 non-null float64
9     8971 non-null float64
10    8971 non-null float64
11    8971 non-null float64
12    8971 non-null float64
13    8971 non-null float64
14    8970 non-null float64
15    8970 non-null float64
16    8969 non-null float64
17    8947 non-null float64
18    8188 non-null float64
19    1904 non-null float64
20    35 non-null float64
21    9 non-null float64
22    4 non-null float64
23    3 non-null float64
24    2 non-null float64
25    2 non-null float64
26    2 non-null float64
27    1 non-null float64
28    1 non-null float64
29    1 non-null float64
30    1 non-null float64
31    1 non-null float64
32    1 non-null float64
33    1

In [18]:
accYDf.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,...,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42
0,-0.39891,-1.83615,-1.17784,-1.57057,-0.64358,0.37380,1.50949,1.95836,1.61260,0.38598,-0.47274,-0.40889,-1.15461,-1.18423,-1.08534,1.73656,1.60372,-0.61712,0.53289,0.47221,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.05398,-0.28840,-1.59451,-1.12244,-0.42300,0.48899,0.54436,0.58867,0.60878,0.60955,0.30371,-0.63048,-1.14801,-1.20960,-0.55401,0.10836,0.25519,-0.24027,0.80372,0.45904,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.25695,-0.06710,-0.29890,0.58825,1.86493,-0.74524,-3.40034,-2.19935,0.89133,0.88155,0.12292,-0.28479,-0.45773,-2.25096,-2.18389,-1.80846,-0.84924,-0.12781,0.84771,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.86116,0.41307,-0.43575,-0.45591,0.40136,1.24040,0.34871,-1.39129,-1.25099,0.74597,0.61204,-0.76690,-0.50632,0.11588,-0.93399,-2.80052,-2.36305,0.13212,1.25486,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-1.12954,-4.36667,-4.39744,-3.45672,-0.16053,6.10793,5.89761,0.12116,-4.53063,-5.12621,-6.06235,-3.59347,1.32365,1.91694,1.85529,1.83659,1.10147,0.29886,-0.69567,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# Konvertiere Beschleunigungssensordaten Z-Achse zu einem DataFrame
measuredDataList = featuresDf['Z-Achse Beschleunigungswerte in m/s^2'].str.split(" ").tolist()
accList = []
accList.append(pandas.DataFrame(np.array(measuredDataList[0]).astype(float)).transpose())
for x in measuredDataList[1:]:
    accList.append(pandas.DataFrame(np.array(x).astype(float)).transpose())
accZDf = pandas.concat(accList, ignore_index=True)

In [20]:
# Konvertiere Nick Messwerte zu DataFrame
measuredDataList = featuresDf['Nick Messwerte in rad'].str.split(" ").tolist()
pitchList = []
pitchList.append(pandas.DataFrame(np.array(measuredDataList[0]).astype(float)).transpose())
for x in measuredDataList[1:]:
    pitchList.append(pandas.DataFrame(np.array(x).astype(float)).transpose())
nickDf = pandas.concat(pitchList, ignore_index=True)

In [21]:
# Konvertiere Roll Messwerte zu DataFrame
measuredDataList = featuresDf['Roll Messwerte in rad'].str.split(" ").tolist()
rollList = []
rollList.append(pandas.DataFrame(np.array(measuredDataList[0]).astype(float)).transpose())
for x in measuredDataList[1:]:
    rollList.append(pandas.DataFrame(np.array(x).astype(float)).transpose())
rollDf = pandas.concat(rollList, ignore_index=True)

In [22]:
measuredDataList = featuresDf['Gier Messwerte in rad'].str.split(" ").tolist()
azimuthList = []
azimuthList.append(pandas.DataFrame(np.array(measuredDataList[0]).astype(float)).transpose())
for x in measuredDataList[1:]:
    azimuthList.append(pandas.DataFrame(np.array(x).astype(float)).transpose())
azimuthDf = pandas.concat(azimuthList, ignore_index=True)

In [23]:
measuredDataList = featuresDf['Zeitstempel der Messwerte in ns'].str.split(" ").tolist()
timestampsList = []
timestampsList.append(pandas.DataFrame(np.array(measuredDataList[0]).astype(float)).transpose())
for x in measuredDataList[1:]:
    timestampsList.append(pandas.DataFrame(np.array(x).astype(float)).transpose())
sensorTimestampsDf = pandas.concat(timestampsList, ignore_index=True)

In [24]:
accXMean = accXDf.abs().T.mean()
accXMean.head(2)

0    0.543271
1    0.346206
dtype: float64

In [25]:
accYDf.abs().T.mean().head()

0    1.036779
1    0.601754
2    1.111971
3    0.896331
4    2.840986
dtype: float64

In [26]:
# Neue Spalten werden erstellt und Mittelwerte zugewiesen
featuresDf['Mittelwert X-Achse Beschleunigung in m/s^2'] = accXMean
featuresDf['Mittelwert Y-Achse Beschleunigung in m/s^2'] = accYDf.abs().T.mean()
featuresDf['Mittelwert Z-Achse Beschleunigung in m/s^2'] = accZDf.abs().T.mean()
featuresDf['Mittelwert Nick in rad'] = nickDf.abs().T.mean()
featuresDf['Mittelwert Roll in rad'] = rollDf.abs().T.mean()

In [27]:
featuresDf['Mittelwert X-Achse Beschleunigung in m/s^2'].head(2)

0    0.543271
1    0.346206
Name: Mittelwert X-Achse Beschleunigung in m/s^2, dtype: float64

In [28]:
featuresDf.columns

Index(['Zeitstempel in Unixzeit', 'Breitengrad', 'Laengengrad',
       'Geschwindigkeit in km/h', 'X-Achse Beschleunigungswerte in m/s^2',
       'Y-Achse Beschleunigungswerte in m/s^2',
       'Z-Achse Beschleunigungswerte in m/s^2', 'Gier Messwerte in rad',
       'Nick Messwerte in rad', 'Roll Messwerte in rad',
       'Zeitstempel der Messwerte in ns', 'Anzahl der Messwerte',
       'Start des Zeitfensters in ns seit Start der JVM',
       'Start der Messwerterfassung in ns seit Start der JVM',
       'Start der Belichtung in ns seit Start der JVM',
       'Belichtungszeit in ns',
       'Letzter Zeitstempel der Messwerterfassung in ns seit Start der JVM',
       'Stopp der Messwerterfassung in Unixzeit',
       'Mittelwert X-Achse Beschleunigung in m/s^2',
       'Mittelwert Y-Achse Beschleunigung in m/s^2',
       'Mittelwert Z-Achse Beschleunigung in m/s^2', 'Mittelwert Nick in rad',
       'Mittelwert Roll in rad'],
      dtype='object')

Berechnet die Varianz. Dieser Funktion muss als Paramter der Mittelwert (mean) und der DataFrame mit   
den Float Werten übergeben werden, um die Varianz zu berechnen. Als Varianz wird der Durchschnitt der quadrierten  
Differenzen zum Mittelwert bezeichnet.   
Prec.:    
Postc.: Gibt die berechnete Variance als Float zurück oder 0 wenn die Anzahl der Werte <= 0  

In [29]:
def calcVariance(meansDf, dfValues):
    variance = []
    for i, values in dfValues.iterrows():
        sum = 0
        for value in values:
            if math.isnan(value) == False:
                tempDifference = value-meansDf[i]
                sum += tempDifference * tempDifference
        variance.append("{0:.5f}".format(round(sum / values.count(),5)))
    return np.array(variance).astype(float)

Hier wird die Standardabweichung der Varianz berechnet. Dies ist die Wurzel der Varianz.   
Für die Berechnung wird der Absolutwert von der Varianz genommen.    
Prec.:
Postc.:  Standardabweichung wird zurückgegeben.

In [30]:
def calculateStandardDeviation(varianceDf):
    deviation = []
    for v in varianceDf:
        temp = math.sqrt(np.abs(v))
        deviation.append("{0:.5f}".format(round(temp,5)))
    return np.array(deviation).astype(float)

Die Funktion berechnet die Winkeldifferenz in rad zwischen den ersten gemessenen Gierwinkel und den letzten Gierwinkel   
innerhalb einer als Argument übergebenen DataFrames. Der zurückgegebene Radiant    
ist immer positiv und gibt die relative änderung in rad an.      
Prec.:   
Postc.: Radiant berechnet   

In [31]:
def calculateAngelChangeAzimuth(azimuthDf):
    result = []
    for i,values in azimuthDf.iterrows():
        first = values[0]*(180/math.pi)
        last = values.iloc[values.count()-1]*(180/math.pi)
        resultTemp = np.abs(first - last)
        if(resultTemp > 180):
            resultTemp = 360 - resultTemp
        result.append("{0:.5f}".format(round(resultTemp * ( math.pi / 180),5)))
    return np.array(result).astype(float)

In [32]:
featuresDf['Gier-Aenderung in rad'] = calculateAngelChangeAzimuth(azimuthDf)

In [33]:
# Berechnung der Varianzen und hinzufügen der Ergebnisse als neue Spalte in dem DataFrame 
featuresDf['Varianz X-Achse Beschleunigung in m/s^2'] = calcVariance(featuresDf['Mittelwert X-Achse Beschleunigung in m/s^2'],accXDf)
featuresDf['Varianz Y-Achse Beschleunigung in m/s^2'] = calcVariance(featuresDf['Mittelwert Y-Achse Beschleunigung in m/s^2'],accYDf)
featuresDf['Varianz Z-Achse Beschleunigung in m/s^2'] = calcVariance(featuresDf['Mittelwert Z-Achse Beschleunigung in m/s^2'],accZDf)
featuresDf['Varianz Nick in rad'] = calcVariance(featuresDf['Mittelwert Nick in rad'],nickDf)
featuresDf['Varianz Roll in rad'] = calcVariance(featuresDf['Mittelwert Roll in rad'],rollDf)

In [34]:
featuresDf['Varianz X-Achse Beschleunigung in m/s^2'].head()

0    0.24436
1    0.32940
2    1.35824
3    1.42913
4    5.01102
Name: Varianz X-Achse Beschleunigung in m/s^2, dtype: float64

In [35]:
calculateStandardDeviation(featuresDf['Varianz X-Achse Beschleunigung in m/s^2'])

array([0.49433, 0.57393, 1.16544, ..., 0.47966, 0.51355, 0.26406])

In [36]:
# Berechne Standardabweichung
featuresDf['Standardabweichung X-Achse Beschleunigung in m/s^2'] = calculateStandardDeviation(featuresDf['Varianz X-Achse Beschleunigung in m/s^2'])
featuresDf['Standardabweichung Y-Achse Beschleunigung in m/s^2'] = calculateStandardDeviation(featuresDf['Varianz Y-Achse Beschleunigung in m/s^2'])
featuresDf['Standardabweichung Z-Achse Beschleunigung in m/s^2'] = calculateStandardDeviation(featuresDf['Varianz Z-Achse Beschleunigung in m/s^2'])
featuresDf['Standardabweichung Nick in rad'] = calculateStandardDeviation(featuresDf['Varianz Nick in rad'])
featuresDf['Standardabweichung Roll in rad'] = calculateStandardDeviation(featuresDf['Varianz Roll in rad'])

In [37]:
featuresDf['Standardabweichung X-Achse Beschleunigung in m/s^2'].head()

0    0.49433
1    0.57393
2    1.16544
3    1.19546
4    2.23853
Name: Standardabweichung X-Achse Beschleunigung in m/s^2, dtype: float64

In [38]:
accXDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8972 entries, 0 to 8971
Data columns (total 43 columns):
0     8972 non-null float64
1     8972 non-null float64
2     8972 non-null float64
3     8972 non-null float64
4     8972 non-null float64
5     8971 non-null float64
6     8971 non-null float64
7     8971 non-null float64
8     8971 non-null float64
9     8971 non-null float64
10    8971 non-null float64
11    8971 non-null float64
12    8971 non-null float64
13    8971 non-null float64
14    8970 non-null float64
15    8970 non-null float64
16    8969 non-null float64
17    8947 non-null float64
18    8188 non-null float64
19    1904 non-null float64
20    35 non-null float64
21    9 non-null float64
22    4 non-null float64
23    3 non-null float64
24    2 non-null float64
25    2 non-null float64
26    2 non-null float64
27    1 non-null float64
28    1 non-null float64
29    1 non-null float64
30    1 non-null float64
31    1 non-null float64
32    1 non-null float64
33    1

In [39]:
accXDf.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,...,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42
0,0.50696,0.60484,0.60654,-0.06632,-0.28282,0.81578,1.08174,0.49772,0.50552,0.92548,0.47996,-0.16758,-0.57834,-0.40131,-0.15241,0.76687,0.69020,0.50626,0.81881,0.40996,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.14722,-0.18861,-0.22743,-0.35041,-0.03508,-0.24251,-0.22457,-0.77718,-0.85151,-0.69645,-0.08208,0.14894,0.59424,0.47547,0.39577,0.73042,0.53844,0.12437,-0.06898,-0.02445,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.18452,-0.28135,-0.13307,0.20008,0.48192,0.96789,0.42196,-0.91882,-1.88420,-1.17018,-0.18524,0.20430,-0.25019,-0.58315,-0.97210,-1.08406,-0.98975,0.31152,0.98480,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.54282,0.25046,0.00124,-0.22876,0.61386,0.58309,0.42058,-0.19975,-0.81861,-1.95726,-1.28991,0.56173,0.44946,-0.69763,-0.75723,-0.37586,-0.65304,-1.47237,-0.45764,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.82189,-1.31905,-2.08180,-2.75329,-3.12193,-2.92650,0.20248,1.24999,0.54038,0.04931,0.95890,-0.10620,-0.57522,-1.18027,-0.72963,-0.35378,0.52918,1.81779,2.74143,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
# Änderungen in CSV Datei schreiben
featuresDf.set_index('Zeitstempel in Unixzeit', inplace=True)
featuresDf.to_csv('../daten/merkmale.csv')

In [41]:
pandas.read_csv("../daten/merkmale.csv").columns

Index(['Zeitstempel in Unixzeit', 'Breitengrad', 'Laengengrad',
       'Geschwindigkeit in km/h', 'X-Achse Beschleunigungswerte in m/s^2',
       'Y-Achse Beschleunigungswerte in m/s^2',
       'Z-Achse Beschleunigungswerte in m/s^2', 'Gier Messwerte in rad',
       'Nick Messwerte in rad', 'Roll Messwerte in rad',
       'Zeitstempel der Messwerte in ns', 'Anzahl der Messwerte',
       'Start des Zeitfensters in ns seit Start der JVM',
       'Start der Messwerterfassung in ns seit Start der JVM',
       'Start der Belichtung in ns seit Start der JVM',
       'Belichtungszeit in ns',
       'Letzter Zeitstempel der Messwerterfassung in ns seit Start der JVM',
       'Stopp der Messwerterfassung in Unixzeit',
       'Mittelwert X-Achse Beschleunigung in m/s^2',
       'Mittelwert Y-Achse Beschleunigung in m/s^2',
       'Mittelwert Z-Achse Beschleunigung in m/s^2', 'Mittelwert Nick in rad',
       'Mittelwert Roll in rad', 'Gier-Aenderung in rad',
       'Varianz X-Achse Beschleunigung

In [42]:
# Speichert die DatenFrames mit den Messwerten in csv Dateien
accXDf.index.names = ["index"]
accXDf.to_csv('../daten/x-Achse-Beschleunigungswerte.csv')
accYDf.index.names = ["index"]
accYDf.to_csv('../daten/y-Achse-Beschleunigungswerte.csv')
accZDf.index.names = ["index"]
accZDf.to_csv('../daten/z-Achse-Beschleunigungswerte.csv')
nickDf.index.names = ["index"]
nickDf.to_csv('../daten/nick-Messwerte.csv')
rollDf.index.names = ["index"]
rollDf.to_csv('../daten/roll-Messwerte.csv')
azimuthDf.index.names = ["index"]
azimuthDf.to_csv('../daten/azimuth-Messwerte.csv')
sensorTimestampsDf.index.names = ["index"]
sensorTimestampsDf.to_csv('../daten/sensorevent-Zeitstempel.csv')

In [43]:
# Unittests
import unittest

class CalcTest(unittest.TestCase):
    
    
    # Testet die Funktion calcVariance mit einem DataFrame mit den Werten [2.0, 2.0]
    # Der Mittelwert sollte 2.0 sein
    # Das erwartete Ergebnis ist 0.0
    def testCalcVariance2(self):
        meanL = [2.0]
        meanDf = pandas.DataFrame(meanL)
        valuesA = [2.0,2.0]
        valuesDf = pandas.DataFrame([[2.0]])
        npt.assert_almost_equal(calcVariance(meanDf[0],valuesDf), 0.0,2)
        
    # Testet die Funktion calcVariance mit einem DataFrame mit den folgenden Werten:
    # [1.24, 2.5213, 10.434, 42.45, 5.9]
    # Der Mittelwert ist 12.508
    # Die erwartete Varianz ist 234,247016
    def testCalcVariance5(self):
        meanDf = pandas.DataFrame([12.508])
        valuesDf = pandas.DataFrame([[1.24, 2.52, 10.43, 42.45, 5.9]])
        npt.assert_almost_equal(calcVariance(meanDf[0],valuesDf), 234.24702,5)   
    
    # Testet calcVariance mit 0 Datenwerten   
    def testCalcVariance0(self):
        meanDf = pandas.DataFrame([0.0])
        valuesDf = pandas.DataFrame([[0.0]])
        npt.assert_almost_equal(calcVariance(meanDf[0],valuesDf),0.0,2)  
        
    # Testet calcVariance mit negativen Datenwerten   
    def testCalcVarianceNegative(self):
        meanDf = pandas.DataFrame([-24.0])
        valuesDf = pandas.DataFrame([[-2.0,-24.0,-5.0,7.0]])
        npt.assert_almost_equal(calcVariance(meanDf[0],valuesDf),451.5,1) 
        
    # Testet calcVariance mit Muultidimensionalen DataFrame   
    def testCalcVarianceMultidim(self):
        meanDf = pandas.DataFrame([-24.0,0.0])
        valuesDf = pandas.DataFrame([[-2.0,-24.0,-5.0,7.0],[0.0,0.0,0.0,0.0]])
        # Hier wird Numpy Test verwendet um das Ergebnis (zwei Arrays) zu vergleichen
        npt.assert_array_equal(calcVariance(meanDf[0],valuesDf),np.array([ 451.5,0.0])) 
        
    # Dieser Test testet die Funktion calculateStandardDeviation.
    def testCalculateStandardDeviation(self):
        varianceDf = pandas.DataFrame([451.5])
        npt.assert_almost_equal(calculateStandardDeviation(varianceDf[0]),21.24853,5) 
        
    # Testet die Funktion calculateStandardDeviation.
    # Dabei hat das Argument den Wert 0.0.
    def testCalculateStandardDeviationZero(self):
        varianceDf = pandas.DataFrame([0.0])
        npt.assert_almost_equal(calculateStandardDeviation(varianceDf[0]),0.0,2)
    
    #  Testet die Funktion calculateAngelChangeAzimuth mit zwei gleichen rad Werten
    def testCalculateAngelChangeAzimuthEqual(self):
        radiants = pandas.DataFrame([[1.0,1.0]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants),0.0,2)
    
    # Testet ob die Winkeldifferenz korrekt berechnet wurde für 
    # die Radianten -1.0 und 1.0
    def testCalculateAngelChangeAzimuthOne(self):
        radiants = pandas.DataFrame([[-1.0,1.0]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants),2.0,2)  
        
    # Testet ob die Winkeldifferenz korrekt berechnet wurde für 
    # die Radianten 1.0 und -1.0
    def testCalculateAngelChangeAzimuthOne2(self):
        radiants = pandas.DataFrame([[1.0,-1.0]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants),2.0,2)  

    # Testet die Funktion calculateAngelChangeAzimuthValues mit mehreren Werten
    def testCalculateAngelChangeAzimuthValues(self):
        radiants = pandas.DataFrame([[-1.57,1.0,2.0,1.3,1.57]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants),3.14,2)
        
    # Testet ob die Winkeldifferenz korrekt berechnet wurde für 
    # die Radianten 0.05 und 0.06
    def testCalculateAngelChangeAzimuthSmall(self):
        radiants = pandas.DataFrame([[0.01,0.02]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants), 0.01,2)  
        
    # Testet ob die Winkeldifferenz korrekt berechnet wurde mit 
    # negative Radianten 
    def testCalculateAngelChangeAzimuthNegativ(self):
        radiants = pandas.DataFrame([[-2.9,-2.0,-1.28]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants), 1.62,2) 

unittest.main(argv=[''], verbosity=2, exit=False)

testCalcVariance0 (__main__.CalcTest) ... ok
testCalcVariance2 (__main__.CalcTest) ... ok
testCalcVariance5 (__main__.CalcTest) ... ok
testCalcVarianceMultidim (__main__.CalcTest) ... ok
testCalcVarianceNegative (__main__.CalcTest) ... ok
testCalculateAngelChangeAzimuthEqual (__main__.CalcTest) ... ok
testCalculateAngelChangeAzimuthNegativ (__main__.CalcTest) ... ok
testCalculateAngelChangeAzimuthOne (__main__.CalcTest) ... ok
testCalculateAngelChangeAzimuthOne2 (__main__.CalcTest) ... ok
testCalculateAngelChangeAzimuthSmall (__main__.CalcTest) ... ok
testCalculateAngelChangeAzimuthValues (__main__.CalcTest) ... ok
testCalculateStandardDeviation (__main__.CalcTest) ... ok
testCalculateStandardDeviationZero (__main__.CalcTest) ... ok

----------------------------------------------------------------------
Ran 13 tests in 0.025s

OK
